## `RSI & Bolinger Band & MACD`

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

#### Download Data

In [3]:
stock_symbol = "TLSA"
stock_data = yf.download(stock_symbol, period="max")

# Create a Date column
stock_data['Date'] = stock_data.index

# Drop the Date as index
stock_data.reset_index(drop=True, inplace=True)
stock_data.head(5)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,Date
Ticker,TLSA,TLSA,TLSA,TLSA,TLSA,
0,1.670668,2.100840,1.670668,2.100840,87465,2018-11-20
1,2.118848,2.434974,1.838735,1.838735,38984,2018-11-21
2,1.960784,2.200880,1.958784,2.200880,14994,2018-11-23
3,1.676671,1.940776,1.552621,1.940776,24990,2018-11-26
4,1.792717,1.800720,1.700680,1.760704,21991,2018-11-27


#### Clean Data